In [1]:
# Import necessary packages
# For data frames
import pandas as pd
import numpy as np
# For weather forcast
from darksky import forecast
# For dates
from datetime import datetime as dt
from datetime import timedelta
# For holidays
import holidays

In [2]:
# Set parameters for weather extraction
# apikey = load from textfile
phila = [39.9526, -75.1652]
PHILA = apikey, phila[0], phila[1]
us_holidays = holidays.UnitedStates()

In [4]:
# Set dates for weather extraction
start_date = "2015-12-31"
stop_date = "2017-01-01"
start = dt.strptime(start_date, "%Y-%m-%d")
stop = dt.strptime(stop_date, "%Y-%m-%d")
stop-start

datetime.timedelta(days=367)

In [5]:
# initialize structure that we'll populate with data
d = []

In [6]:
# loop through dates and get weather data
# loop through the hours of the day and extract hourly weather data 
while start < stop:
    start = start + timedelta(days=1)
    start_time = start.isoformat()
    pp = forecast(*PHILA, time = start_time)
   
    # check the length of hours
    nHours = 24
    tempHours = len(pp.hourly)
    if tempHours < nHours: 
        nHours = len(pp.hourly)
    
    for i in range(0,nHours): 
        # PULL INTO SEPARATE FUNCTION
        # First check for availability of the data (if value is not aviailable assign None)
        # 1 precipitation intensity
        pInt = None
        try: 
            pInt = pp.hourly[i].precipIntensity
        except(AttributeError):
            print('missingData')
        
        # 2 preciptiation Type
        pType = None
        try: 
            pType = pp.hourly[i].precipType
        except(AttributeError):
            print('missingData')
        
        # 3 precipitation Probability
        pProb = None
        try: 
            pProb = pp.hourly[i].precipProbability
        except(AttributeError):
            print('missingData') 
        
        # 4 temperature
        temp = None
        try: 
            temp = pp.hourly[i].temperature
        except(AttributeError):
            print('missingData') 
        
        # 5 apparent temperature
        appTemp = None
        try: 
            appTemp = pp.hourly[i].apparentTemperature
        except(AttributeError):
            print('missingData') 
        
        # 6 humidity
        humid = None
        try: 
            humid = pp.hourly[i].humidity
        except(AttributeError):
            print('missingData')    
            
        # 7 pressure
        press = None
        try: 
            press = pp.hourly[i].pressure
        except(AttributeError):
            print('missingData')    
                
        # 8 wind speed
        wSpeed = None
        try: 
            wSpeed = pp.hourly[i].windSpeed
        except(AttributeError):
            print('missingData')      
            
        # 9 wind bearing
        wBearing = None
        try: 
            wBearing = pp.hourly[i].windBearing
        except(AttributeError):
            print('missingData')       
            
        # 10 visibiilty
        visib = None
        try: 
            visib = pp.hourly[i].visibility
        except(AttributeError):
            print('missingData')  
        
        d.append({'date_time': start_time,
            'day': start.day,
            'month': start.month,
            'hour': start + timedelta(hours = i),
            'weekday': start.weekday(), 
            'is_holiday': start in us_holidays,
            'which_holiday':  holidays.US(years = start.year).get(start),     
            'precipIntensity': pInt, 
            'precipProbability': pProb,
            'precipType': pType, 
            'temperature': temp,  
            'apparentTemperature': appTemp,  
            'humidity': humid,  
            'pressure': press,  
            'windSpeed': wSpeed,  
            'windBearing': wBearing, 
            'visibility': visib})
# variables that are there but I am not using them: uv_index, icon, cloudCover, summary, dewPoint

missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
miss

missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
miss

missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
miss

missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
miss

missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
miss

...

missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
miss

missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
miss

missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
miss

missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
miss

missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData
missingData


In [7]:
df = pd.DataFrame(d)
weather_filename = start_date
df.to_csv("Weather2016.csv")